In [2]:
import torch
import numpy as np

In [7]:
from pyfillet import TextEmbedder

data = 0
with open('TrainingData/schizophrenia.txt') as f:
    data = f.readlines()

text = ''
for string in data:
    text += string

embedder = TextEmbedder()
embeddings = embedder(text)
print(len(embeddings))

69940


In [67]:
bar = {}
for obj in embeddings:
    bar[obj[0]] = torch.tensor(obj[1])

In [68]:
class TextGenerator(torch.nn.Module):
    def __init__(self, embedding_dict):
        super(TextGenerator, self).__init__()
        self.block = torch.nn.Sequential(
            torch.nn.Linear(300*2, 256),  # Eats 2 vec-words, return vec-word
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 300))
        self.embedding_dict = embedding_dict
        
    def the_most_similar(self, embedded_vector):  # tries to find the most similar word with respect to vec-word
        word = ''
        least_distance = 1000000.0
        for key, value in self.embedding_dict.items():
            if least_distance >= float((embedded_vector - value).dot(embedded_vector - value)):
                word = key
        return word
        
    def forward(self, prev_words):
        input = torch.tensor([])
        for word in prev_words:
            input = torch.cat((input, self.embedding_dict[word]))
        x = self.block(input)
        return x
    
    def generate (self, length = 30, prefix = None):
        if prefix is None:
            prefix = [self.embedding_dict[np.random.randint(1, 3000)], self.embedding_dict[np.random.randint(1, 3000)]]
        text = ''
        for word in prefix:
            text += word + ' '
        for i in range(length):
            next_word_vec = self.forward(prefix)
            next_word = self.the_most_similar(next_word_vec)
            prefix.append(next_word)
            prefix.pop(0)
            text += next_word + ' '
        return text

Набросок генеративной модельки, тут нет обучения, его можно сделать через mse, так как после эмбеддинга норма в пространстве представления
слов обрело смысл схожести слов, так что спомощью батч+AdaM и MSE можно оптимизировать модель, разбив текст окном в 3 слова (2 на вход один на сравнение). Скорее всего получится не слишком хорошо, потому что всего 3 скрытых слоя и без фокусов со скипами, но вера есть...